In [53]:
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import numpy as np
import os
import rdkit

import pandas as pd
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem

from ipywidgets import interact

fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [66]:
# import pickle
# file = open(fp_path, 'rb')
# data = pickle.load(file)
# file.close()


In [484]:
fn = 'ic_50/herg.sdf'
df_ic50 = PandasTools.LoadSDF(fn, embedProps=True, molColName=None, smilesName='smiles')


directory = "fingerprints/herg/"
df = DataFrame()
for i, filename in enumerate(os.listdir(directory)):
    molecule_id = filename[11:-16]
    file_id = filename[5:-16]

    fpt_filepath = directory+filename

    fp = plf.Fingerprint.from_pickle(fpt_filepath)
    fprint_df = fp.to_dataframe()
    df_row = df_ic50[df_ic50["Molecule ChEMBL ID"] == molecule_id]
    if len(df_row[~df_row["Standard Value"].isna()]) > 0:

        mol_filepath = "docked/herg/herg_"+file_id+".mol2"
        output = !./smina.osx -r receptors/hERG.pdb -l {mol_filepath} --center_x 141.475  --center_y 141.38225  --center_z 166.96675 --score_only
        aff = re.findall(r'Affinity:\s*(\-?[\d\.]+)', '\n'.join(output))
        intr_en = re.findall(r'Intramolecular energy:\s*(\-?[\d\.]+)', '\n'.join(output))
        aff = [float(i) for i in aff]
        intr_en = [float(i) for i in intr_en]
        a_max = max(aff)
        i_e_min = min(intr_en)

        if df_row.iloc[0]["Standard Value"] != "nan":
            fprint_df["affinity"] = a_max
            fprint_df["intra_en"] = i_e_min
            fprint_df["IC_50"] = df_row.iloc[0]["Standard Value"]
            fprint_df["CHEMBL_ID"] = molecule_id
            df = pd.concat([df, fprint_df])

    

# df = df[df['IC_50'].notna()]
# df = df.fillna(0.0)
# for column in df.columns:
#     print(column)
#     if column[0] != "CHEMBL_ID":
#         df[column] = df[column].astype(float)

# df2 = df.reset_index()
# df2 = df2.drop(columns=["Frame"])

# df2.to_csv("herg_data.csv")

In [537]:
df = df[df['IC_50'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6401 entries, 0 to 6400
Data columns (total 92 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   (UNL1, TYR652.A, Hydrophobic)  6401 non-null   float64
 1   (UNL1, TYR652.A, VdWContact)   6401 non-null   float64
 2   (UNL1, SER624.B, VdWContact)   6401 non-null   float64
 3   (UNL1, TYR652.B, Hydrophobic)  6401 non-null   float64
 4   (UNL1, TYR652.B, PiStacking)   6401 non-null   float64
 5   (UNL1, TYR652.C, Hydrophobic)  6401 non-null   float64
 6   (UNL1,   K1101.C, VdWContact)  6401 non-null   float64
 7   (UNL1, TYR652.D, Hydrophobic)  6401 non-null   float64
 8   (UNL1, PHE656.D, Hydrophobic)  6401 non-null   float64
 9   (affinity, , )                 6401 non-null   float64
 10  (intra_en, , )                 6401 non-null   float64
 11  (IC_50, , )                    6401 non-null   float64
 12  (CHEMBL_ID, , )                6401 non-null   o

In [538]:
df = df[df['IC_50'].notna()]
df = df.fillna(0.0)
for column in df.columns:
    #print(column)
    if column[0] != "CHEMBL_ID":
        df[column] = df[column].astype(float)

df = df.reset_index()
df = df.drop(columns=["Frame"])
df.info()
df.to_csv("herg_data.csv")

KeyError: 'Frame'

In [488]:
herg = pd.read_csv('herg_data.csv')
herg.head()

In [490]:

herg = herg[2:]
Y = herg["IC_50"]
herg = herg.drop(columns = ["CHEMBL_ID", "IC_50", "ligand"])

herg.head()

,UNL1,UNL1.1,UNL1.2,UNL1.3,UNL1.4,UNL1.5,UNL1.6,UNL1.7,UNL1.8,affinity,...,UNL1.78,UNL1.79,UNL1.80,UNL1.81,UNL1.82,UNL1.83,UNL1.84,UNL1.85,UNL1.86,UNL1.87
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-7.18486,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-7.37992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-7.28949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-6.27381,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,-8.12051,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [493]:
herg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6401 entries, 2 to 6402
Data columns (total 90 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UNL1      6401 non-null   object 
 1   UNL1.1    6401 non-null   object 
 2   UNL1.2    6401 non-null   object 
 3   UNL1.3    6401 non-null   object 
 4   UNL1.4    6401 non-null   object 
 5   UNL1.5    6401 non-null   object 
 6   UNL1.6    6401 non-null   object 
 7   UNL1.7    6401 non-null   object 
 8   UNL1.8    6401 non-null   object 
 9   affinity  6401 non-null   float64
 10  intra_en  6401 non-null   float64
 11  UNL1.9    6401 non-null   object 
 12  UNL1.10   6401 non-null   object 
 13  UNL1.11   6401 non-null   object 
 14  UNL1.12   6401 non-null   object 
 15  UNL1.13   6401 non-null   object 
 16  UNL1.14   6401 non-null   object 
 17  UNL1.15   6401 non-null   object 
 18  UNL1.16   6401 non-null   object 
 19  UNL1.17   6401 non-null   object 
 20  UNL1.18   6401 non-null   obje

In [492]:
herg = herg.drop(columns=["ligand"])
herg.head()

KeyError: "['ligand'] not found in axis"

In [436]:
herg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6401 entries, 2 to 6402
Data columns (total 89 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UNL1      6401 non-null   object 
 1   UNL1.1    6401 non-null   object 
 2   UNL1.2    6401 non-null   object 
 3   UNL1.3    6401 non-null   object 
 4   UNL1.4    6401 non-null   object 
 5   UNL1.5    6401 non-null   object 
 6   UNL1.6    6401 non-null   object 
 7   UNL1.7    6401 non-null   object 
 8   UNL1.8    6401 non-null   object 
 9   affinity  6401 non-null   float64
 10  UNL1.9    6401 non-null   object 
 11  UNL1.10   6401 non-null   object 
 12  UNL1.11   6401 non-null   object 
 13  UNL1.12   6401 non-null   object 
 14  UNL1.13   6401 non-null   object 
 15  UNL1.14   6401 non-null   object 
 16  UNL1.15   6401 non-null   object 
 17  UNL1.16   6401 non-null   object 
 18  UNL1.17   6401 non-null   object 
 19  UNL1.18   6401 non-null   object 
 20  UNL1.19   6401 non-null   obje

In [600]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# scaler = StandardScaler()

# X_train, X_test, y_train, y_test = train_test_split(
#      herg, Y, test_size=0.25, random_state=42)

# X_train["affinity"] = scaler.fit_transform(np.array(X_train["affinity"]).reshape(-1, 1))
# X_test["affinity"] = scaler.transform(np.array(X_test["affinity"]).reshape(-1, 1))
# X_train["intra_en"] = scaler.fit_transform(np.array(X_train["intra_en"]).reshape(-1, 1))
# X_test["intra_en"] = scaler.transform(np.array(X_test["intra_en"]).reshape(-1, 1))

# pca = PCA(n_components=30) #
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)


# #print(X_train.head())

# X_test, X_train = tf.convert_to_tensor(X_test, float), tf.convert_to_tensor(X_train, float)

# y_train_log = tf.math.log(y_train)
# y_test_log = tf.math.log(y_test)

In [643]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
scaler = StandardScaler()



X_train, X_test, y_train, y_test = train_test_split(
     herg, Y, test_size=0.25, random_state=42)


X_train["affinity"] = scaler.fit_transform(np.array(X_train["affinity"]).reshape(-1, 1))
X_test["affinity"] = scaler.transform(np.array(X_test["affinity"]).reshape(-1, 1))
X_train["intra_en"] = scaler.fit_transform(np.array(X_train["intra_en"]).reshape(-1, 1))
X_test["intra_en"] = scaler.transform(np.array(X_test["intra_en"]).reshape(-1, 1))

print(X_train.head())

X_test, X_train = tf.convert_to_tensor(X_test, float), tf.convert_to_tensor(X_train, float)

y_train_log = tf.math.log(y_train)
y_test_log = tf.math.log(y_test)


     UNL1 UNL1.1 UNL1.2 UNL1.3 UNL1.4 UNL1.5 UNL1.6 UNL1.7 UNL1.8  affinity   
1788  1.0    0.0    0.0    1.0    0.0    1.0    0.0    1.0    0.0 -0.708703  \
5664  1.0    0.0    0.0    0.0    0.0    0.0    1.0    1.0    0.0 -0.250685   
2630  1.0    1.0    1.0    0.0    0.0    0.0    1.0    1.0    0.0 -0.042438   
4085  1.0    1.0    0.0    1.0    1.0    0.0    0.0    1.0    1.0  0.512854   
4151  1.0    0.0    0.0    1.0    0.0    0.0    1.0    1.0    0.0 -0.972078   

      ...  UNL1.78 UNL1.79 UNL1.80 UNL1.81 UNL1.82 UNL1.83 UNL1.84 UNL1.85   
1788  ...      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  \
5664  ...      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2630  ...      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4085  ...      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4151  ...      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

     UNL1.86 UNL1.87  
1788     0.0     0.0  
5664     0

In [644]:
sc_y = StandardScaler()
y_train_log_scaled = tf.convert_to_tensor(sc_y.fit_transform(np.array(y_train_log).reshape(-1, 1)), float)
y_test_log_scaled = tf.convert_to_tensor(sc_y.transform(np.array(y_test_log).reshape(-1, 1)), float)

In [670]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 18).fit(X_train, y_train_log_scaled)

prediction = rf.predict(X_test)
mse = mean_squared_error(y_test_log_scaled, prediction)
rmse = mse**.5
print(mse)
print(rmse)
r2 = r2_score(y_test_log_scaled, prediction)
print(r2)

/var/folders/fz/k5cf128j3qn__2x3wf8_t4tw0000gn/T/ipykernel_41851/2754066033.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 18).fit(X_train, y_train_log_scaled)


0.9503608553575102
0.9748645317978853
0.034675276541445754


In [679]:
from sklearn.model_selection import GridSearchCV


grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [15,16,17,18,19,20, 21, 22],
    'random_state' : [18, 20, 48]
}

CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
CV_rfr.fit(np.array(X_train), np.array(y_train_log_scaled))


/Users/mariolawozniak/miniconda3/envs/mldd23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/mariolawozniak/miniconda3/envs/mldd23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/mariolawozniak/miniconda3/envs/mldd23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/mariolawozniak/miniconda3/envs/ml

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [15, 16, 17, 18, 19, 20, 21, 22],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [200, 300, 400, 500],
                         'random_state': [18, 20, 48]})

In [681]:
print(CV_rfr.best_params_, CV_rfr.best_estimator_)

{'max_depth': 18, 'max_features': 'sqrt', 'n_estimators': 300, 'random_state': 18} RandomForestRegressor(max_depth=18, max_features='sqrt', n_estimators=300,
                      random_state=18)


In [682]:
rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 18, random_state = 18)  # najlepszy model dotychczas
rf.fit(X_train, y_train_log_scaled)
# Predict on test data
prediction = rf.predict(X_test)
# Compute mean squared error
mse = mean_squared_error(y_test_log_scaled, prediction)
# Print results
print(mse)
r2 = r2_score(y_test_log_scaled, prediction)
print(r2)

/var/folders/fz/k5cf128j3qn__2x3wf8_t4tw0000gn/T/ipykernel_41851/2337002720.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train_log_scaled)


0.8969905249132651
0.0888859473477488


In [591]:
rf.fit(X_train, y_train_log)

RandomForestRegressor(max_depth=3, max_features='log2', n_estimators=400,
                      random_state=18)

In [568]:
prediction = rf.predict(X_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(mse)
print(rmse)
r2 = r2_score(y_test, prediction)
print(r2)

180164539897551.9
13422538.504230559
-0.0007232382887021327


In [683]:
import tensorflow
from tensorflow import keras
seq = keras.Sequential([
    keras.layers.Dense(64, input_shape=X_train.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("leaky_relu"),
    keras.layers.Dropout(0.7),
    keras.layers.Dense(64),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("leaky_relu"),
    keras.layers.Dropout(0.7),
    keras.layers.Dense(32),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("leaky_relu"),
    keras.layers.Dropout(0.7),
    #keras.layers.Dense(128),
    #keras.layers.BatchNormalization(),
    #keras.layers.Activation("leaky_relu"),
    keras.layers.Dense(1, activation = "leaky_relu")
]
)

In [684]:
seq.compile(loss='mean_squared_error',
                optimizer=tensorflow.keras.optimizers.Adam(0.001))

In [647]:
seq.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_77 (Dense)            (None, 64)                5824      
                                                                 
 batch_normalization_56 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 activation_56 (Activation)  (None, 64)                0         
                                                                 
 dropout_43 (Dropout)        (None, 64)                0         
                                                                 
 dense_78 (Dense)            (None, 64)                4160      
                                                                 
 batch_normalization_57 (Bat  (None, 64)               256       
 chNormalization)                                    

In [711]:
history = seq.fit(X_train, y_train_log_scaled, epochs = 10, batch_size=64, verbose=1, validation_data=(X_test, y_test_log_scaled), validation_freq=1)

Epoch 1/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9472 - val_loss: 0.9386
Epoch 2/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9467 - val_loss: 0.9379
Epoch 3/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9372 - val_loss: 0.9362
Epoch 4/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9363 - val_loss: 0.9395
Epoch 5/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9336 - val_loss: 0.9395
Epoch 6/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9473 - val_loss: 0.9403
Epoch 7/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9312 - val_loss: 0.9436
Epoch 8/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9307 - val_loss: 0.9418
Epoch 9/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9340 - val_loss: 0.9408
Epoch 10/10
75/75 [==============================] - 0s 2ms/step - loss: 0.9461 - val_loss: 0.9386


In [712]:
prediction = seq.predict(X_test)
mse = mean_squared_error(y_test_log_scaled, prediction)
rmse = mse**.5
print(mse)
print(rmse)
r2 = r2_score(y_test_log_scaled, prediction)
print(r2)

51/51 [==============================] - 0s 567us/step
0.93855065
0.9687882385090437
0.046671380511297045


In [557]:
from sklearn.svm import SVR
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train_log_scaled)

prediction = regressor.predict(X_test)
mse = mean_squared_error(y_test_log_scaled, prediction)
rmse = mse**.5
print(mse)
print(rmse)
r2 = r2_score(y_test_log_scaled, prediction)
print(r2)

/Users/mariolawozniak/miniconda3/envs/mldd23/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9369089781883772
0.9679405860838656
0.048338932786571975


,UNL1,UNL1.1,UNL1.2,UNL1.3,UNL1.4,UNL1.5,UNL1.6,UNL1.7,UNL1.8,UNL1.9,...,UNL1.104,UNL1.105,UNL1.106,UNL1.107,UNL1.108,UNL1.109,UNL1.110,UNL1.111,UNL1.112,UNL1.113
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [483]:
import re

directory = "docked/herg/"
for i, filename in enumerate(os.listdir(directory)[0:1]):
    path = directory+filename
    #output = 
    output = !./smina.osx -r receptors/hERG.pdb -l {path} --center_x 141.475  --center_y 141.38225  --center_z 166.96675 --score_only
    aff = re.findall(r'Affinity: \s*(\-?[\d\.]+)', '\n'.join(output))
    intr_en = re.findall(r'Intramolecular energy:\s*(\-?[\d\.]+)', '\n'.join(output))
    aff = [float(i) for i in aff]
    intr_en = [float(i) for i in intr_en]
    a_max = max(aff)
    i_e_min = min(intr_en)
    print(output)

['   _______  _______ _________ _        _______ ', '  (  ____ \\(       )\\__   __/( (    /|(  ___  )', '  | (    \\/| () () |   ) (   |  \\  ( || (   ) |', '  | (_____ | || || |   | |   |   \\ | || (___) |', '  (_____  )| |(_)| |   | |   | (\\ \\) ||  ___  |', '        ) || |   | |   | |   | | \\   || (   ) |', '  /\\____) || )   ( |___) (___| )  \\  || )   ( |', '  \\_______)|/     \\|\\_______/|/    )_)|/     \\|', '', '', 'smina is based off AutoDock Vina. Please cite appropriately.', '', 'Weights      Terms', '-0.035579    gauss(o=0,_w=0.5,_c=8)', '-0.005156    gauss(o=3,_w=2,_c=8)', '0.840245     repulsion(o=0,_c=8)', '-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)', '-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)', '1.923        num_tors_div', '', '## Name gauss(o=0,_w=0.5,_c=8) gauss(o=3,_w=2,_c=8) repulsion(o=0,_c=8) hydrophobic(g=0.5,_b=1.5,_c=8) non_dir_h_bond(g=-0.7,_b=0,_c=8) num_tors_div', 'Affinity: -8.09516 (kcal/mol)', 'Intramolecular energy: -0.25850', 'Term values, befo